<a href="https://colab.research.google.com/github/hpatel1398/het-patel/blob/master/NNDLDemo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.0MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=1,validation_split=0.1)

Trial 5 Complete [00h 02m 19s]
val_accuracy: 0.8891666531562805

Best val_accuracy So Far: 0.8920000195503235
Total elapsed time: 00h 14m 04s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        13856     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1734768   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 1,751,002
Trainable params: 1,751,002
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 122s 72ms/step - loss: 0.2643 - accuracy: 0.9027 - val_loss: 0.2621 - val_accuracy: 0.9037
Epoch 5/10
1688/1688 [==============================] - 118s 70ms/step - loss: 0.2057 - accuracy: 0.9227 - val_loss: 0.2695 - val_accuracy: 0.9077
Epoch 6/10
1688/1688 [==============================] - 120s 71ms/step - loss: 0.1609 - accuracy: 0.9395 - val_loss: 0.2606 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 119s 70ms/step - loss: 0.1219 - accuracy: 0.9542 - val_loss: 0.2743 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 119s 70ms/step - loss: 0.0901 - accuracy: 0.9674 - val_loss: 0.3069 - val_accuracy: 0.9123
Epoch 9/10
1688/1688 [==============================] - 118s 70ms/step - loss: 0.0714 - accuracy: 0.9737 - val_loss: 0.3184 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 117s 70ms/step - loss: 0.0516 - accuracy: 0.9815 - val_loss: 